In [35]:

import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: abhilash001vj. Use `wandb login --relogin` to force relogin


True

# Introduction

We are building an image classifer that can classify website screeshots into the following categories.
- Marketplace
- Forum
- General (neither market nor forum)

In [17]:
labels = ["marketplace","forum","general"]

In [4]:
os.listdir("./")

['data_process_and_train.ipynb', 'webs-te-image-classifier.ipynb']

In [6]:
import os
import glob

folder_0_images = glob.glob("../data/circl-ail-dataset-01/folder_0/*.png")

In [19]:

folder_1_images = glob.glob("../data/circl-ail-dataset-01/folder_1_SECONDPASS/*.png")

In [20]:
len(folder_1_images), len(folder_0_images)

(4000, 4000)

In [18]:
import json
label_file = "../data/circl-ail-dataset-01/labels_0.json"
with open(label_file) as f:
    folder_0_labels = json.load(f)
label_file = "../data/circl-ail-dataset-01/labels_1.json"
with open(label_file) as f:
    folder_1_labels = json.load(f)

In [49]:
import shutil

data_dir = "../data/website_images_1000/"
common_class = "general"
label_count = {}
max_count = 1000
os.makedirs(os.path.join(data_dir,common_class), exist_ok=True)
os.makedirs(data_dir, exist_ok=True)
for img_path in folder_0_images:
    filename  = os.path.basename(img_path)
    label_list = folder_0_labels[filename]
    found_label = False
    for label in labels:
        os.makedirs(os.path.join(data_dir,label), exist_ok=True)
        if label in " ".join(label_list) and label_count.get(label, 0) < max_count:
            shutil.copy(img_path, os.path.join(data_dir,label,filename))
            found_label = True
            label_count[label] = label_count.get(label, 0) + 1
            break
    if not found_label and label_count.get(common_class, 0) < max_count:
        label_count[common_class] = label_count.get(common_class, 0 )+ 1
        shutil.copy(img_path, os.path.join(data_dir,common_class,filename))


os.makedirs(os.path.join(data_dir,common_class), exist_ok=True)
os.makedirs(data_dir, exist_ok=True)
for img_path in folder_1_images:
    filename  = os.path.basename(img_path)
    label_list = folder_1_labels[filename]
    found_label = False
    for label in labels:
        os.makedirs(os.path.join(data_dir,label), exist_ok=True)
        if label in " ".join(label_list) and label_count.get(label, 0) < max_count:
            shutil.copy(img_path, os.path.join(data_dir,label,filename))
            found_label = True
            label_count[label] = label_count.get(label, 0) + 1
            break
    if not found_label and label_count.get(common_class, 0) < max_count:
        label_count[common_class] = label_count.get(common_class, 0) + 1
        shutil.copy(img_path, os.path.join(data_dir,common_class,filename))
        

In [46]:
" ".join(label_list)

'dark-web:topic="drugs-narcotics"'

In [50]:
import os
import pandas as pd

def get_image_count(root_dir):
    label_dirs = [dir for dir in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, dir))]

    count_dict = {}
    total_count = 0

    # Iterate through each directory (label)
    for dir in label_dirs:
        dir_path = os.path.join(root_dir, dir)

        # Count only image files, assuming they have extensions .jpg, .png, .jpeg, .tiff, .bmp, .gif
        image_files = [f for f in os.listdir(dir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]
        image_count = len(image_files)

        count_dict[dir] = image_count
        total_count += image_count

    count_dict['total'] = total_count

    return count_dict

root_dir = "../data/website_images_1000/"  # replace with your data directory
counts = get_image_count(root_dir)
# Create a dataframe from counts dictionary and print
df = pd.DataFrame(list(counts.items()), columns=['Label', 'Image Count'])
print(df)

         Label  Image Count
0        forum         1000
1      general         1000
2  marketplace         1000
3        total         3000


In [ ]:
# https://wandb.ai/mostafaibrahim17/ml-articles/reports/A-Gentle-Introduction-to-Image-Classification--VmlldzozNDI4MjQ4
# https://wandb.ai/stacey/mendeleev/reports/Tables-Tutorial-Visualize-Data-for-Image-Classification--VmlldzozNjE3NjA
# https://colab.research.google.com/github/wandb/examples/blob/master/colabs/datasets-predictions/Image_Classification_with_Tables.ipynb#scrollTo=ios7mrU2BcIF

In [51]:
import os
from random import shuffle
import numpy as np
DATA_SRC = "../data/website_images/"
# source directory for all raw data
SRC = DATA_SRC
PREFIX = "website_screeshots" # convenient for tracking local data
PROJECT_NAME = "Website Classification"
IMAGES_PER_LABEL = 0
# number of images per class label
# the total number of images is 10X this (10 classes)
TOTAL_IMAGES = 3000 # IMAGES_PER_LABEL * 10

# Step 1: Upload raw data

In [52]:
# if this is a substantially new dataset, give it a new name
# this will create a whole new placeholder (Artifact) for this dataset
# instead of just incrementing a version of the old dataset
RAW_DATA_AT = "_".join([PREFIX, "raw_data", str(TOTAL_IMAGES)])
run = wandb.init(project=PROJECT_NAME, job_type="upload")
# create an artifact for all the raw data
raw_data_at = wandb.Artifact(RAW_DATA_AT, type="raw_data")

# SRC_DIR contains 10 folders, one for each of 10 class labels
# each folder contains images of the corresponding class
labels = os.listdir(SRC)
for l in labels:
  imgs_per_label = os.path.join(SRC, l)
  if os.path.isdir(imgs_per_label):
    # filter out "DS_Store"
    imgs = [i for i in os.listdir(imgs_per_label) if not i.startswith(".DS")]
    # randomize the order
    shuffle(imgs)
    img_file_ids = imgs[:IMAGES_PER_LABEL]
    for f in img_file_ids:
      file_path = os.path.join(SRC, l, f)
      # add file to artifact by full path
      raw_data_at.add_file(file_path, name=l + "/" + f)

# save artifact to W&B
run.log_artifact(raw_data_at)
run.finish()

# Step 2: Split raw data to prepare for training


In [53]:
{"train" : ( TOTAL_IMAGES * 0.8 ),  "val" : ( TOTAL_IMAGES * 0.1 ), "test": ( TOTAL_IMAGES * 0.1 ) }

{'train': 2400.0, 'val': 300.0, 'test': 300.0}

In [54]:
BALANCED_SPLITS = {"train" : ( TOTAL_IMAGES * 0.8 ),  "val" : ( TOTAL_IMAGES * 0.1 ), "test": ( TOTAL_IMAGES * 0.1 ) }
# if this is a substantially different dataset, give it a new name
# this will create a whole new placeholder (Artifact) for this split
# instead of just incrementing a version of the old data split
SPLIT_DATA_AT = "_".join([PREFIX, "80-10-10", str(TOTAL_IMAGES)])
run = wandb.init(project=PROJECT_NAME, job_type="data_split")

# create balanced train, val, test splits
# each count is the number of images per label
SPLIT_COUNTS = BALANCED_SPLITS

# find the most recent ("latest") version of the full raw data
# you can of course pass around programmatic aliases and not string literals
# note: RAW_DATA_AT is defined in the previous cell—if you're running
# just this step, you may need to hardcode it
data_at = run.use_artifact(RAW_DATA_AT + ":latest")
# download it locally (for illustration purposes/across hardware; you can
# also sync/version artifacts by reference)
data_dir = data_at.download()

data_split_at = wandb.Artifact(SPLIT_DATA_AT, type="balanced_data")

# create a table with columns we want to track/compare
preview_dt = wandb.Table(columns=["id", "image", "label", "split"])

labels = os.listdir(data_dir)
for l in labels:
  if l.startswith("."): # skip non-label file
    continue
  imgs_per_label = os.listdir(os.path.join(data_dir, l))
  shuffle(imgs_per_label)
  start_id = 0
  for split, count in SPLIT_COUNTS.items():
    # take a subset
    split_imgs = imgs_per_label[start_id:start_id+count]
    for img_file in split_imgs:
      f_id = img_file.split(".")[0]
      full_path = os.path.join(data_dir, l, img_file)
      # add file to artifact by full path
      # note: pass the label to the name parameter to retain it in
      # the data structure 
      data_split_at.add_file(full_path, name = os.path.join(split, l, img_file))
      # add a preview of the image
      # if SIZE == "LARGE": # skip for the largest dataset for efficiency
      #   continue
      if split != "test":
        preview_dt.add_data(f_id, wandb.Image(full_path), l, split)
      else:
        # pretend we have unlabeled test data
        # (replace "unknown" with l if you'd like to keep the labels :)
        preview_dt.add_data(f_id, wandb.Image(full_path), "unknown", split)
    start_id += count

# log artifact to W&B
data_split_at.add(preview_dt, "data_split")
run.log_artifact(data_split_at)
run.finish()

FileNotFoundError: [WinError 3] The system cannot find the path specified: '.\\artifacts\\website_screeshots_raw_data_3000-v0'

# Step 3: Train with artifacts and save model


In [55]:
# EXPERIMENT CONFIG
#------------------------
# Core globals to modify
NUM_EPOCHS = 5 # set low for demo purposes, try 3, or 5, or as many as you like


# optional globals to modify
# set to a custom name to help keep your experiments organized
RUN_NAME = "base_model" 
# change this if you'd like start a new set of comparable Tables
# (only Tables logged to the same key can be compared)
VAL_TABLE_NAME = "predictions" 

# hyperparams set low for demo/training speed
# if you set these higher, be mindful of how many items are in
# the dataset artifacts you chose by setting the SIZE at the top
NUM_TRAIN = BALANCED_SPLITS["train"]
NUM_VAL = BALANCED_SPLITS["val"]

# enforced max for this is ceil(NUM_VAL/batch_size)
NUM_LOG_BATCHES = 16

# ARTIFACTS CONFIG
#------------------------
# training data artifact to load
TRAIN_DATA_AT = PREFIX + "_80-10-10_" + str(TOTAL_IMAGES)

# model name
# if you want to train a sufficiently different model, give this a new name
# to start a new lineage for the model, instead of just incrementing the
# version of the old model
MODEL_NAME = "iv3_finetuned"

# folder in which to save the final, trained model
# if you want to train a sufficiently different model, give this a new name
# to start a new lineage for the model, instead of just incrementing the
# version of the old model
SAVE_MODEL_DIR = "finetune_iv3_keras"

from tensorflow.keras.applications.inception_v3 import InceptionV3
# from tensorflow.keras.applications.resnet import InceptionV3

from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from wandb.keras import WandbCallback

# experiment configuration saved to W&B
CFG = {
  "num_train" : NUM_TRAIN,
  "num_val" : NUM_VAL,
  "num_classes" : 10,
  "fc_size" : 1024,
  "epochs" : NUM_EPOCHS,
  "batch_size" : 32,

  # inceptionV3 settings
  "img_width" : 299,
  "img_height": 299
}

# number of validation data batches to log/use when computing metrics
# at the end of each epoch
max_log_batches = int(np.ceil(float(CFG["num_val"])/float(CFG["batch_size"])))
# change this min to max to log ALL the available images to a Table
CFG["num_log_batches"] = min(max_log_batches, NUM_LOG_BATCHES)

def finetune_inception_model(fc_size, num_classes):
  """Load InceptionV3 with ImageNet weights, freeze it,
  and attach a finetuning top for this classification task"""
  # load InceptionV3 as base
  base = InceptionV3(weights="imagenet", include_top="False")
  # freeze base layers
  for layer in base.layers:
    layer.trainable = False
  x = base.get_layer('mixed10').output 

  # attach a fine-tuning layer
  x = GlobalAveragePooling2D()(x)
  x = Dense(fc_size, activation='relu')(x)
  guesses = Dense(num_classes, activation='softmax')(x)

  model = Model(inputs=base.input, outputs=guesses)
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model

def train():
  """ Main training loop which freezes the InceptionV3 layers of the model
  and only trains the new top layers on the new data. A subsequent training
  phase might unfreeze all the layers and finetune the whole model on the new data""" 
  run = wandb.init(project=PROJECT_NAME, name=RUN_NAME, job_type="train", config=CFG)
  cfg = wandb.config

  # locate and download training and validation data
  data_at = TRAIN_DATA_AT + ":latest"
  data = run.use_artifact(data_at, type="balanced_data")
  data_dir = data.download()
  train_dir = os.path.join(data_dir, "train")
  val_dir = os.path.join(data_dir, "val")

  # create train and validation data generators
  train_datagen = ImageDataGenerator(
      rescale=1. / 255,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True)
  val_datagen = ImageDataGenerator(rescale=1. / 255)

  train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(cfg.img_width, cfg.img_height),
    batch_size=cfg.batch_size,
    class_mode='categorical')

  val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(cfg.img_width, cfg.img_height),
    batch_size=cfg.batch_size,
    class_mode='categorical',
    shuffle=False)

  # instantiate model and callbacks
  model = finetune_inception_model(cfg.fc_size, cfg.num_classes)
  callbacks = [WandbCallback(), ValLog(val_generator, cfg.num_log_batches)]

  # train!
  model.fit(
    train_generator,
    steps_per_epoch = cfg.num_train // cfg.batch_size,
    epochs=cfg.epochs,
    validation_data=val_generator,
    callbacks = callbacks,
    validation_steps = cfg.num_val // cfg.batch_size)

  # save trained model as artifact
  trained_model_artifact = wandb.Artifact(
            MODEL_NAME, type="model",
            description="finetuned inception v3",
            metadata=dict(cfg))
  
  model.save(SAVE_MODEL_DIR)
  trained_model_artifact.add_dir(SAVE_MODEL_DIR)
  run.log_artifact(trained_model_artifact)
  run.finish()

class ValLog(Callback):
  """ Custom callback to log validation images
  at the end of each training epoch"""
  def __init__(self, generator=None, num_log_batches=1):
    self.generator = generator
    self.num_batches = num_log_batches
    # store full names of classes
    self.flat_class_names = [k for k, v in generator.class_indices.items()]

  def on_epoch_end(self, epoch, logs={}):
    # collect validation data and ground truth labels from generator
    val_data, val_labels = zip(*(self.generator[i] for i in range(self.num_batches)))
    val_data, val_labels = np.vstack(val_data), np.vstack(val_labels)

    # use the trained model to generate predictions for the given number
    # of validation data batches (num_batches)
    val_preds = self.model.predict(val_data)
    true_ids = val_labels.argmax(axis=1)
    max_preds = val_preds.argmax(axis=1)

    # log validation predictions alongside the run
    columns=["id", "image", "guess", "truth"]
    for a in self.flat_class_names:
      columns.append("score_" + a)
    predictions_table = wandb.Table(columns = columns)
    
    # log image, predicted and actual labels, and all scores
    for filepath, img, top_guess, scores, truth in zip(self.generator.filenames,
                                                       val_data, 
                                                       max_preds, 
                                                       val_preds,
                                                       true_ids):
      img_id = filepath.split('/')[-1].split(".")[0]
      row = [img_id, wandb.Image(img), 
             self.flat_class_names[top_guess], self.flat_class_names[truth]]
      for s in scores.tolist():
        row.append(np.round(s, 4))
      predictions_table.add_data(*row)
    wandb.run.log({VAL_TABLE_NAME : predictions_table})

In [56]:
train()

CommError: Project abhilash001vj/Website Classification does not contain artifact: "website_screeshots_80-10-10_3000:latest"

# Step 4: Load model for inference


In [ ]:
# optional globals to modify
# set to a custom name to help keep your experiments organized
RUN_NAME = "" 
# change this if you'd like start a new set of comparable Tables
# (only Tables logged to the same key can be compared)
TEST_TABLE_NAME = "test_results" 

from tensorflow import keras
from tensorflow.keras.preprocessing import image
import numpy as np
import os

MODEL_NAME = "iv3_finetuned"
# location of test data from our original split
# should match SPLIT_DATA_AT
TEST_DATA_AT = "_".join([PREFIX, "80-10-10", str(TOTAL_IMAGES)])

run = wandb.init(project=PROJECT_NAME, job_type="inference", name=RUN_NAME)
model_at = run.use_artifact(MODEL_NAME + ":latest")
model_dir = model_at.download()
print("model: ", model_dir)
model = keras.models.load_model(model_dir)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# download latest version of test data
test_data_at = run.use_artifact(TEST_DATA_AT + ":latest")
test_dir = test_data_at.download()
test_dir += "/test/"

class_names = ["Animalia", "Amphibia", "Arachnida", "Aves", "Fungi", 
               "Insecta", "Mammalia", "Mollusca", "Plantae", "Reptilia"]

# load test images
imgs = []
filenames = []
class_labels = os.listdir(test_dir)
truth = []
for l in class_labels:
  if l.startswith("."):
    continue
  imgs_per_class = os.listdir(os.path.join(test_dir, l))
  for img in imgs_per_class:
    # track the image id
    filenames.append(img.split(".")[0])
    truth.append(l)
    img_path = os.path.join(test_dir, l, img)
    img = image.load_img(img_path, target_size=(299, 299))
    img = image.img_to_array(img)
    # don't forget to rescale test images to match the range of inputs
    # to the network
    img = np.expand_dims(img/255.0, axis=0)
    imgs.append(img)

# predict on test data and bin predictions by guessed label 
preds = {}
imgs = np.vstack(imgs)
classes = model.predict(imgs, batch_size=32)
for c in classes:
  class_id = np.argmax(c)
  if class_id in preds:
    preds[class_id] += 1
  else:
    preds[class_id] = 1

# log inference results as a Table to the run workspace
columns=["id", "image", "guess", "truth"]
for a in class_names:
  columns.append("score_" + a)
test_dt = wandb.Table(columns = columns)

# store all the scores for each image
for img_id, i, t, c in zip(filenames, imgs, truth, classes):
  guess = class_names[np.argmax(c)]
  row = [img_id, wandb.Image(i), guess, t]
  for c_i in c.tolist():
    row.append(np.round(c_i, 4))
  test_dt.add_data(*row)
  
run.log({TEST_TABLE_NAME : test_dt})
print("Quick distribution of predicted classes: ")
print(preds)
run.finish()

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torchvision.models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
model.fc = nn.Linear(num_ftrs, 3)

model_ft = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

data_dir = "../data/website_images_1000/"
transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.ImageFolder( root=data_dir, transform=transform)
train_loader = torch.utils.data.DataLoader(
	train_dataset,
	batch_size=64,
	shuffle=True,
	num_workers=4
      )




c:\Users\abhil\miniconda3\envs\sd\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\abhil\miniconda3\envs\sd\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\abhil\miniconda3\envs\sd\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
train_dataset.classes

['forum', 'general', 'marketplace']

In [5]:
train_dataset.class_to_idx

{'forum': 0, 'general': 1, 'marketplace': 2}

In [ ]:
train_dataset.idx

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
for epoch in range(10):
	for images, labels in train_loader:
		images, labels = images.to(device), labels.to(device)
		optimizer.zero_grad()
		output = model(images)
		loss = criterion(output, labels)
		loss.backward()
		optimizer.step()


	print("Epoch: {} Loss: {:.4f}".format(epoch, loss.item()))


Epoch: 0 Loss: 0.9253
Epoch: 1 Loss: 0.6486
Epoch: 2 Loss: 0.5883
Epoch: 3 Loss: 0.3971
Epoch: 4 Loss: 0.9180
Epoch: 5 Loss: 0.5661
Epoch: 6 Loss: 0.2214
Epoch: 7 Loss: 0.5264
Epoch: 8 Loss: 0.2841
Epoch: 9 Loss: 0.3814


In [3]:
PATH = './website_classifier.pth'
torch.save(model.state_dict(), PATH)

In [ ]:
dataiter = iter(testloader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

In [ ]:
net = None
net.load_state_dict(torch.load(PATH))
outputs = net(images)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')